In [ ]:
import torch
import torchaudio
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, SeamlessM4Tv2Model
from llama_cpp import Llama
from langdetect import detect
from gtts import gTTS
import io
from pydub import AudioSegment
import numpy as np
import pandas as pd
import time
import os

c:\Users\moham\anaconda3\envs\my_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\moham\anaconda3\envs\my_env\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:

input_text = [
    ["English",    "Detect an obstacle, avoid it, and then change color to green."],
    ["French",     "Détectez un obstacle, évitez-le, puis changez de couleur pour passer au vert."],
    ["German",     "Erkennen Sie ein Hindernis, umfahren Sie es und wechseln Sie dann die Farbe auf Grün."],
    ["Spanish",    "Detecten un obstáculo, evítenlo y luego cambien de color a verde."],
    ["Italian",    "Rilevate un ostacolo, evitatelo e poi cambiate il colore in verde."],
    ["Dutch",      "Detecteer een obstakel, vermijd het en verander vervolgens de kleur naar groen."],
    ["Polish",     "Wykryjcie przeszkodę, ominijcie ją, a następnie zmieńcie kolor na zielony."],
    ["Portuguese", "Detectem um obstáculo, evitem-no e depois mudem a cor para verde."],
    ["Swedish",    "Upptäck ett hinder, undvik det och byt sedan färg till grönt."],


    ["English",    "Wander randomly until a target is detected, approach it, and signal achievement by changing color to red."],
    ["French",     "Errez de manière aléatoire jusqu'à ce qu'une cible soit détectée, approchez-vous et signalez la réussite en changeant la couleur en rouge."],
    ["German",     "Streift zufällig umher, bis ein Ziel erkannt wird, nähert euch ihm an und signalisiert den Erfolg, indem ihr die Farbe auf Rot ändert."],
    ["Spanish",    "Deambulen aleatoriamente hasta que se detecte un objetivo, acérquense a él y señalen el logro cambiando el color a rojo."],
    ["Italian",    "Vagate casualmente finché non viene rilevato un bersaglio, avvicinatevi e segnalate il successo cambiando il colore in rosso."],
    ["Dutch",      "Zwerft willekeurig totdat een doel wordt gedetecteerd, nader het en geef het bereiken aan door de kleur te veranderen naar rood."],
    ["Polish",     "Włóczcie się losowo, aż wykryty zostanie cel, podejdźcie do niego i zasygnalizujcie osiągnięcie, zmieniając kolor na czerwony."],
    ["Portuguese", "Vaguem aleatoriamente até que um alvo seja detectado, aproximem-se dele e sinalizem a conquista mudando a cor para vermelho."],
    ["Swedish",    "Vandra slumpmässigt tills ett mål upptäcks, närma er det och signalera framgång genom att byta färg till rött."],


    ["English",    "Check if the path is clear, and form a line at the center."],
    ["French",     "Vérifiez si le chemin est libre, puis formez une ligne au centre."],
    ["German",     "Überprüft, ob der Weg frei ist, und bildet eine Linie in der Mitte."],
    ["Spanish",    "Comprueben si el camino está despejado y formen una línea en el centro."],
    ["Italian",    "Verificate che il percorso sia libero e formate una linea al centro."],
    ["Dutch",      "Controleer of het pad vrij is en vorm een lijn in het midden."],
    ["Polish",     "Sprawdźcie, czy droga jest wolna, i utwórzcie linię pośrodku."],
    ["Portuguese", "Verifiquem se o caminho está livre e formem uma linha no centro."],
    ["Swedish",    "Kontrollera om vägen är fri och bilda en linje i mitten."],


    ["English",    "Find the goal, signal success by changing color to red, and align movement with other swarm agents."],
    ["French",     "Trouvez l'objectif, signalez la réussite en changeant la couleur en rouge et alignez votre mouvement avec les autres agents de l'essaim."],
    ["German",     "Findet das Ziel, signalisiert den Erfolg, indem ihr die Farbe auf Rot ändert, und stimmt eure Bewegung mit anderen Schwarmagenten ab."],
    ["Spanish",    "Encuentren la meta, señalen el éxito cambiando el color a rojo y alineen su movimiento con otros agentes del enjambre."],
    ["Italian",    "Trovate l'obiettivo, segnalate il successo cambiando il colore in rosso e allineate i vostri movimenti con gli altri agenti dello sciame."],
    ["Dutch",      "Vind het doel, geef succes aan door de kleur te veranderen naar rood, en stem je beweging af op andere zwermagenten."],
    ["Polish",     "Znajdźcie cel, zasygnalizujcie sukces, zmieniając kolor na czerwony, i dostosujcie ruch do innych agentów roju."],
    ["Portuguese", "Encontrem o objetivo, sinalizem o sucesso mudando a cor para vermelho e alinhem o movimento com outros agentes do enxame."],
    ["Swedish",    "Hitta målet, signalera framgång genom att byta färg till rött och anpassa rörelsen med andra svärmagenter."],


    ["English",    "Detect the target, and freeze movement upon reaching it."],
    ["French",     "Détectez la cible et arrêtez tout mouvement en l'atteignant."],
    ["German",     "Erkennt das Ziel und stoppt die Bewegung, sobald ihr es erreicht."],
    ["Spanish",    "Detecten el objetivo y detengan todo movimiento al alcanzarlo."],
    ["Italian",    "Rilevate l'obiettivo e bloccate il movimento una volta raggiunto."],
    ["Dutch",      "Detecteer het doel en stop de beweging zodra het is bereikt."],
    ["Polish",     "Wykryjcie cel i zatrzymajcie ruch po jego osiągnięciu."],
    ["Portuguese", "Detectem o alvo e congelem o movimento ao alcançá-lo."],
    ["Swedish",    "Upptäck målet och frys rörelsen när ni når det."],
]




In [8]:
# Function to update results
def update_results(results_df, output_path, model, language, input_text, translated_text, model_time, time):
    # Create a new DataFrame if results_df is None
    if results_df is None:
        results_df = pd.DataFrame(columns=["Model", "Language", "Input", "Output", "Model time", "Time (s)"])
    
    # Create a new DataFrame for the current entry
    new_entry = pd.DataFrame([{
        "Model": model,
        "Language": language,
        "Input": input_text,
        "Output": translated_text,
        "Model time": model_time,
        "Time (s)": time,
    }])

    # Concatenate the new entry with the existing DataFrame
    results_df = pd.concat([results_df, new_entry], ignore_index=True)

    # Save the DataFrame to the specified output path
    results_df.to_csv(output_path, index=False)
    return results_df



In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "facebook/seamless-m4t-v2-large"
model = SeamlessM4Tv2Model.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True).to(device)
processor = AutoProcessor.from_pretrained(model_id)

# processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
# model = .from_pretrained("facebook/seamless-m4t-v2-large")


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.81it/s]


In [51]:

def seamless_model(audio_file):
    try:
        # 1) Set device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # 2) Read raw MP3 bytes into a waveform tensor
        audio_file.seek(0)
        mp3_bytes = io.BytesIO(audio_file.read())
        waveform, sr = torchaudio.load(mp3_bytes, format="mp3")  # shape: [channels, samples]

        # 3) Convert to mono
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # 4) Resample to 16 kHz if needed
        target_sr = 16000
        if sr != target_sr:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
            waveform = resampler(waveform)

        # 5) Convert to float32 NumPy array in [–1, +1]
        audio_array = waveform.squeeze(0).numpy().astype(np.float32)

        # 6) Prepare inputs for the model
        audio_inputs = processor(audios=audio_array, sampling_rate=target_sr, return_tensors="pt")
        audio_inputs = {k: v.to(device) for k, v in audio_inputs.items()}

        # 7) Inference
        start_time = time.time()
        output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)
        end_time = time.time()

        # 8) Decode
        translated_text = processor.batch_decode(output_tokens.sequences, skip_special_tokens=True)[0]
        return translated_text, (end_time - start_time)

    except Exception as e:
        return f"Error during audio translation: {e}", None




In [52]:
# Function to process text and translate it using SeamlessM4T
def process_text_seamless(text):
    try:
        # Detect language
        language = detect(text)

        # Convert text to speech and save as an in-memory buffer
        tts = gTTS(text=text, lang=language)
        audio_buffer = io.BytesIO()
        tts.write_to_fp(audio_buffer)
        audio_buffer.seek(0)

        # Translate the audio
        translated_text, model_time = seamless_model(audio_buffer)
        return translated_text, model_time

    except Exception as e:
        return f"Error during text processing: {e}", None

In [53]:
output_file = r"./results/seamless-m4t-v2-large-results.csv"
existing_df = pd.read_csv(output_file) if os.path.exists(output_file) else None


for language, text in input_text:
    print(text)
    start_time = time.time()
    translated_text, model_time = process_text_seamless(str(text))
    end_time = time.time()

    # Append a new entry to the results DataFrame
    existing_df = update_results(  
        results_df=existing_df,
        output_path=output_file,
        model="seamless-m4t-v2-large",
        language=language,
        input_text=text,
        translated_text=translated_text,
        model_time=model_time,
        time=end_time - start_time 
    )

print("seamless-m4t-v2-large done")

Detect an obstacle, avoid it, and then change color to green.


C:\Users\moham\AppData\Local\Temp\ipykernel_1692\1039760007.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_entry], ignore_index=True)


Détectez un obstacle, évitez-le, puis changez de couleur pour passer au vert.
Erkennen Sie ein Hindernis, umfahren Sie es und wechseln Sie dann die Farbe auf Grün.
Detecten un obstáculo, evítenlo y luego cambien de color a verde.
Rilevate un ostacolo, evitatelo e poi cambiate il colore in verde.
Detecteer een obstakel, vermijd het en verander vervolgens de kleur naar groen.
Wykryjcie przeszkodę, ominijcie ją, a następnie zmieńcie kolor na zielony.
Detectem um obstáculo, evitem-no e depois mudem a cor para verde.
Upptäck ett hinder, undvik det och byt sedan färg till grönt.
Wander randomly until a target is detected, approach it, and signal achievement by changing color to red.
Errez de manière aléatoire jusqu'à ce qu'une cible soit détectée, approchez-vous et signalez la réussite en changeant la couleur en rouge.
Streift zufällig umher, bis ein Ziel erkannt wird, nähert euch ihm an und signalisiert den Erfolg, indem ihr die Farbe auf Rot ändert.
Deambulen aleatoriamente hasta que se de

### Text Processing models

In [4]:
# Load EuroLLM models
model_path_9b = r"G:\Inventors Hub Projects\SwarmChat\models\EuroLLM-9B-Instruct-Q4_K_M.gguf"
llm_9b = Llama(model_path=model_path_9b, n_ctx=1024)

# Function to process text using EuroLLM
def process_text_EuroLLM_9B(text, llm):
    input_prompt = f"""
    <|im_start|>system
    <|im_end|>
    <|im_start|>user
    Translate the following text to English:
    Text: {text}
    English: 
    <|im_end|>
    <|im_start|>assistant
    """
    output = llm(input_prompt, max_tokens=1024, temperature=0)
    # print('\n\nOutput: ',output,'\n\n')
    full_response = output.get("choices", [{}])[0].get("text", "").strip()
    # print('\n\nResponse: ',full_response,'\n\n')
    return full_response


llama_model_loader: loaded meta data with 41 key-value pairs and 381 tensors from G:\Inventors Hub Projects\SwarmChat\models\EuroLLM-9B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = EuroLLM 9B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = EuroLLM
llama_model_loader: - kv   5:                         general.size_label str              = 9B
llama_model_loader: - kv   6:                            general.license str              = apache-2.0
llama_model

In [9]:
output_file = r"./results/EuroLLM-9B-results.csv"
existing_df = pd.read_csv(output_file) if os.path.exists(output_file) else None



for language, text in input_text:
    start_time = time.time()
    translated_text = process_text_EuroLLM_9B(str(text), llm_9b)
    end_time = time.time()

    # Append a new entry to the results DataFrame
    existing_df = update_results(
        results_df=existing_df,
        output_path=output_file,
        model="EuroLLM-9B",
        language=language,
        input_text=text,
        translated_text=translated_text,
        model_time=None,
        time=end_time - start_time 
    )

print("EuroLLM-9B-results updated_df")

Llama.generate: 55 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   11502.09 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    16 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3532.97 ms /    17 tokens
C:\Users\moham\AppData\Local\Temp\ipykernel_9424\1039760007.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_entry], ignore_index=True)
Llama.generate: 26 prefix-match hit, remaining 36 prompt tokens to eval
llama_perf_context_print:        load time

EuroLLM-9B-results updated_df
